In [1]:
import pandas as pd
import os
import re

To begin, I loop through my directory of act data and collect all of it into a single dataframe. Unnecessary columns are removed after the CRB column index.

In [3]:
#directory path
directory = os.path.expanduser('~/source_data/iowa_act/Iowa_ACT_Data')
#list to store df copies
df_list = []

#loop through excel files in directory
for file in os.listdir(directory):
    #find excel files
    if file.endswith('.xlsx'):

        #create file path for each file
        path = os.path.join(directory, file)
        
        #read in file to a dataframe
        df = pd.read_excel(path, skiprows=2)
        
        #append dataframe to list
        df_list.append(df.copy())

#merge all dfs together
merged_act_data = pd.concat(df_list, ignore_index=True)
column_index = merged_act_data.columns.get_loc('CRB % All Four')

#remove uncessary columns
merged_act_data = merged_act_data.iloc[:, :column_index + 1]

For each school, we see a set of act metrics for a given graduating year.

In [3]:
#ensure correct merging
merged_act_data.head()

,Analysis Level,District Name,ACT Dist Code,HS Name,ACT HS Code,Grad Year,N,Avg Eng,Avg Math,Avg Reading,Avg Sci,Avg Comp,CRB % Eng,CRB % Math,CRB % Reading,CRB % Sci,CRB % All Four
0,District,ADAIR-CASEY CMTY SCH DISTRICT,167342,All Schools,999999.0,2014.0,19,20.5,21.2,20.7,21.6,21.2,68,53,53,37,32
1,District,ADAIR-CASEY CMTY SCH DISTRICT,167342,All Schools,999999.0,2015.0,Small N,Small N,Small N,Small N,Small N,Small N,Small N,Small N,Small N,Small N,Small N
2,District,ADAIR-CASEY CMTY SCH DISTRICT,167342,All Schools,999999.0,2016.0,17,18.9,19.4,20.2,19.8,19.7,59,41,35,12,6
3,District,ADAIR-CASEY CMTY SCH DISTRICT,167342,All Schools,999999.0,2017.0,Small N,Small N,Small N,Small N,Small N,Small N,Small N,Small N,Small N,Small N,Small N
4,District,ADAIR-CASEY CMTY SCH DISTRICT,167342,All Schools,999999.0,2018.0,Small N,Small N,Small N,Small N,Small N,Small N,Small N,Small N,Small N,Small N,Small N


In [4]:
#ensure conversion worked properly
print(merged_act_data.dtypes)

Analysis Level     object
District Name      object
ACT Dist Code      object
HS Name            object
ACT HS Code       float64
Grad Year         float64
N                  object
Avg Eng            object
Avg Math           object
Avg Reading        object
Avg Sci            object
Avg Comp           object
CRB % Eng          object
CRB % Math         object
CRB % Reading      object
CRB % Sci          object
CRB % All Four     object
dtype: object


For many columns, there are rows with a 'Small N' value in what should be numeric columns. Since dashboarding is the goal, we want to be as accurate as possible. Therefore, we will not include rows with this value. This may mean that there will be yearly gaps for certain schools.

In [5]:

def remove_rows_with_small_n(row):

    '''
    The combined dataframe contains many 'Small N' values with variable spacing between the Small and N. This function
    works by fist creating a string representation of the row, then determining whether there is a variation of 'Small N'
    present. If so, remove that row.
    '''
    ##create a string representation of row
    row_string = ' '.join(map(str, row))

    #check if row has a Small N variable present
    if re.search(r'Small\s+N', row_string, re.IGNORECASE):
        return False  #don't keep row if Small N in row
    else:
        return True 

#apply function across data
merged_act_data = merged_act_data[merged_act_data.apply(remove_rows_with_small_n, axis=1)]


In [6]:
merged_act_data.head()

,Analysis Level,District Name,ACT Dist Code,HS Name,ACT HS Code,Grad Year,N,Avg Eng,Avg Math,Avg Reading,Avg Sci,Avg Comp,CRB % Eng,CRB % Math,CRB % Reading,CRB % Sci,CRB % All Four
0,District,ADAIR-CASEY CMTY SCH DISTRICT,167342,All Schools,999999.0,2014.0,19,20.5,21.2,20.7,21.6,21.2,68,53,53,37,32
2,District,ADAIR-CASEY CMTY SCH DISTRICT,167342,All Schools,999999.0,2016.0,17,18.9,19.4,20.2,19.8,19.7,59,41,35,12,6
5,District,ADEL-DE SOTO-MINBURN CMTY SD,167226,All Schools,999999.0,2014.0,75,23.5,23.7,24.3,23.4,23.9,93,69,65,63,47
6,District,ADEL-DE SOTO-MINBURN CMTY SD,167226,All Schools,999999.0,2015.0,79,23,23.4,24.2,23.6,23.7,85,63,67,62,49
7,District,ADEL-DE SOTO-MINBURN CMTY SD,167226,All Schools,999999.0,2016.0,78,22.7,23.5,24.3,23.6,23.7,83,67,69,62,46


There are a handful of columns that are meaningless for this project. A small number of rows also contain district codes that are non-numeric. Both sets are removed.

In [7]:
#drop unnecessary columns and duplicates
merged_act_data = merged_act_data.drop(['Analysis Level', 'HS Name', 'ACT HS Code'], axis=1)
merged_act_data = merged_act_data.drop_duplicates()
#remove weird row where District code is a combination of nums and characters
merged_act_data = merged_act_data[merged_act_data['District Name'].str.strip().str.upper() != 'IOWA DEPT OF EDUCATION']

Inspecting the data again, we have a much more palatable dataset. However, what should be numeric columns are still of type object so we will need to perform a type converstion. The 'ACT District Code' and 'N' columns should be converted to int as well.

In [8]:
merged_act_data.head()

,District Name,ACT Dist Code,Grad Year,N,Avg Eng,Avg Math,Avg Reading,Avg Sci,Avg Comp,CRB % Eng,CRB % Math,CRB % Reading,CRB % Sci,CRB % All Four
0,ADAIR-CASEY CMTY SCH DISTRICT,167342,2014.0,19,20.5,21.2,20.7,21.6,21.2,68,53,53,37,32
2,ADAIR-CASEY CMTY SCH DISTRICT,167342,2016.0,17,18.9,19.4,20.2,19.8,19.7,59,41,35,12,6
5,ADEL-DE SOTO-MINBURN CMTY SD,167226,2014.0,75,23.5,23.7,24.3,23.4,23.9,93,69,65,63,47
6,ADEL-DE SOTO-MINBURN CMTY SD,167226,2015.0,79,23,23.4,24.2,23.6,23.7,85,63,67,62,49
7,ADEL-DE SOTO-MINBURN CMTY SD,167226,2016.0,78,22.7,23.5,24.3,23.6,23.7,83,67,69,62,46


In [9]:
merged_act_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5261 entries, 0 to 12125
Data columns (total 14 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   District Name   5260 non-null   object 
 1   ACT Dist Code   5260 non-null   object 
 2   Grad Year       5260 non-null   float64
 3   N               5260 non-null   object 
 4   Avg Eng         5260 non-null   object 
 5   Avg Math        5260 non-null   object 
 6   Avg Reading     5260 non-null   object 
 7   Avg Sci         5260 non-null   object 
 8   Avg Comp        5260 non-null   object 
 9   CRB % Eng       5260 non-null   object 
 10  CRB % Math      5260 non-null   object 
 11  CRB % Reading   5260 non-null   object 
 12  CRB % Sci       5260 non-null   object 
 13  CRB % All Four  5260 non-null   object 
dtypes: float64(1), object(13)
memory usage: 616.5+ KB


In [10]:
#act data columns are currently object type. We need to convert them into something workable
#let's first grab the columns needed to convert
columns_to_convert = merged_act_data.columns[4:]

#convert act data columns into numeric
for col in columns_to_convert:
    merged_act_data[col] = pd.to_numeric(merged_act_data[col], errors='coerce')

In [13]:
merged_act_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5261 entries, 0 to 12125
Data columns (total 14 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   District Name   5260 non-null   object 
 1   ACT Dist Code   5260 non-null   object 
 2   Grad Year       5260 non-null   float64
 3   N               5260 non-null   object 
 4   Avg Eng         5260 non-null   float64
 5   Avg Math        5260 non-null   float64
 6   Avg Reading     5260 non-null   float64
 7   Avg Sci         5260 non-null   float64
 8   Avg Comp        5260 non-null   float64
 9   CRB % Eng       5260 non-null   float64
 10  CRB % Math      5260 non-null   float64
 11  CRB % Reading   5260 non-null   float64
 12  CRB % Sci       5260 non-null   float64
 13  CRB % All Four  5260 non-null   float64
dtypes: float64(11), object(3)
memory usage: 616.5+ KB


My first attempt to convert the columns failed due to the presence of a single row of NA values. I'm just going to drop this row for now. Below is the try catch block that caught the error.

In [23]:
#empty list to store rows where type conversion occurs
rows_with_type_fail = []

#iterate through every row in the df
for index, row in merged_act_data.iterrows():
    try:
        #attempt to perform type conversion
        merged_act_data.at[index, 'ACT Dist Code'] = int(row['ACT Dist Code'])
        merged_act_data.at[index, 'N'] = int(row['N'])
    except ValueError:
        #if type conversion rows, append row to list
        rows_with_type_fail.append(row)

#append all rows where type conversion failed
rows_with_type_fail_df = pd.concat(problematic_rows, axis=1).T

In [24]:
rows_with_type_fail_df.head()

,District Name,ACT Dist Code,Grad Year,N,Avg Eng,Avg Math,Avg Reading,Avg Sci,Avg Comp,CRB % Eng,CRB % Math,CRB % Reading,CRB % Sci,CRB % All Four
7716,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [25]:
# Drop rows with NaN values from merged_act_data
merged_act_data.dropna(inplace=True)


In [26]:
#convert 'ACT District Code' and 'N' columns to integers
merged_act_data['ACT Dist Code'] = merged_act_data['ACT Dist Code'].astype(int)
merged_act_data['N'] = merged_act_data['N'].astype(int)

In [27]:
#ensure conversion worked properly
print(merged_act_data.dtypes)

District Name      object
ACT Dist Code       int64
Grad Year         float64
N                   int64
Avg Eng           float64
Avg Math          float64
Avg Reading       float64
Avg Sci           float64
Avg Comp          float64
CRB % Eng         float64
CRB % Math        float64
CRB % Reading     float64
CRB % Sci         float64
CRB % All Four    float64
dtype: object



There are inconsistencies in the District Name column. For a single school district with a fixed 'ACT Dist Code', as there could be slightly different naming for the same District Name from year to year. This will be a problem for our dashboard. To demonstrate how inconsistent the naming is I will create a frequency table for names per district code and put it in descending order. Examining the second entry of 167736 (Rock Valley CSD), we see several distinct names, including ROCK VALLEY SCHOOL DISTRICT and ROCK VALLEY CMTY SCH DISTRICT. This is really annoying and I'm not happy with whoever entered this data, but I will fix it. Also observe that we will have to drop duplicates again because of the inconsitent naming.


In [28]:


#create frequency table of names per district code
dist_name_freq_table = merged_act_data.groupby('ACT Dist Code')['District Name'].nunique().reset_index()
dist_name_freq_table = dist_name_freq_table.rename(columns={'District Name': 'Frequency'})
#sort district codes with the most distinct names
dist_name_freq_table = dist_name_freq_table.sort_values(by='Frequency', ascending=False)

dist_name_freq_table.head()

,ACT Dist Code,Frequency
138,167414,4
259,167736,3
2,167008,3
307,167862,3
65,167208,3


In [29]:
#I bet there isn't anything fun to do in rock valley anyway
rock_valley_data = merged_act_data[merged_act_data['ACT Dist Code'] == 167736]
rock_valley_data.head(30)

,District Name,ACT Dist Code,Grad Year,N,Avg Eng,Avg Math,Avg Reading,Avg Sci,Avg Comp,CRB % Eng,CRB % Math,CRB % Reading,CRB % Sci,CRB % All Four
1140,ROCK VALLEY SCHOOL DISTRICT,167736,2014.0,26,22.3,22.3,23.6,22.7,22.9,81.0,65.0,65.0,50.0,31.0
1141,ROCK VALLEY SCHOOL DISTRICT,167736,2015.0,33,21.7,22.5,22.9,23.2,22.8,79.0,58.0,58.0,58.0,42.0
1142,ROCK VALLEY SCHOOL DISTRICT,167736,2016.0,42,22.0,21.2,22.1,22.5,22.0,76.0,52.0,60.0,45.0,36.0
1143,ROCK VALLEY SCHOOL DISTRICT,167736,2017.0,30,21.6,20.8,22.8,21.9,21.9,80.0,37.0,57.0,43.0,23.0
1144,ROCK VALLEY SCHOOL DISTRICT,167736,2018.0,31,19.6,19.3,20.8,20.1,20.0,71.0,32.0,52.0,26.0,13.0
2700,ROCK VALLEY SCHOOL DISTRICT,167736,2012.0,32,19.8,20.4,20.4,21.9,20.8,72.0,41.0,56.0,34.0,22.0
2701,ROCK VALLEY SCHOOL DISTRICT,167736,2013.0,34,21.0,21.1,23.0,22.5,22.0,76.0,44.0,65.0,47.0,41.0
2702,ROCK VALLEY SCHOOL DISTRICT,167736,2014.0,26,22.3,22.3,23.6,22.7,22.9,81.0,65.0,65.0,50.0,31.0
2703,ROCK VALLEY SCHOOL DISTRICT,167736,2015.0,33,21.7,22.5,22.9,23.2,22.8,79.0,58.0,58.0,58.0,42.0
2704,ROCK VALLEY SCHOOL DISTRICT,167736,2016.0,42,22.0,21.2,22.1,22.5,22.0,76.0,52.0,60.0,45.0,36.0


In order to harmonize the district name column, we will determine the name that appears in the majority for each district code. Once we have done that, we will convert non-majority names to the majority name. This ensures that the least amount of data is changed and will allow is to search by name in our dashboard.

In [30]:
#first we create a df to store the majority name for each district code
majority_names_df = merged_act_data.groupby('ACT Dist Code')['District Name'].agg(lambda x: x.mode().iat[0]).reset_index()
#rename columns
majority_names_df.rename(columns={'District Name': 'Majority Name'}, inplace=True)

#using the District Code column merge, this data frame into the act_data column
merged_act_data = pd.merge(merged_act_data, majority_names_df, on='ACT Dist Code', how='left')

#for each row, check if the District Name matches the majority name. If it doesn't, change it
merged_act_data['District Name'] = merged_act_data.apply(
    lambda row: row['Majority Name'] if row['District Name'] != row['Majority Name'] else row['District Name'],
    axis=1
)

#we don't need the majority name column anymore, so we drop it
merged_act_data.drop('Majority Name', axis=1, inplace=True)

#droping duplicated rows after naming reset
merged_act_data = merged_act_data.drop_duplicates()

In [31]:
#Our dataframe now looks nicer than the nicest part of rock valley
rock_valley_data = merged_act_data[merged_act_data['ACT Dist Code'] == 167736]
rock_valley_data.head(30)

,District Name,ACT Dist Code,Grad Year,N,Avg Eng,Avg Math,Avg Reading,Avg Sci,Avg Comp,CRB % Eng,CRB % Math,CRB % Reading,CRB % Sci,CRB % All Four
1077,ROCK VALLEY SCHOOL DISTRICT,167736,2014.0,26,22.3,22.3,23.6,22.7,22.9,81.0,65.0,65.0,50.0,31.0
1078,ROCK VALLEY SCHOOL DISTRICT,167736,2015.0,33,21.7,22.5,22.9,23.2,22.8,79.0,58.0,58.0,58.0,42.0
1079,ROCK VALLEY SCHOOL DISTRICT,167736,2016.0,42,22.0,21.2,22.1,22.5,22.0,76.0,52.0,60.0,45.0,36.0
1080,ROCK VALLEY SCHOOL DISTRICT,167736,2017.0,30,21.6,20.8,22.8,21.9,21.9,80.0,37.0,57.0,43.0,23.0
1081,ROCK VALLEY SCHOOL DISTRICT,167736,2018.0,31,19.6,19.3,20.8,20.1,20.0,71.0,32.0,52.0,26.0,13.0
2554,ROCK VALLEY SCHOOL DISTRICT,167736,2012.0,32,19.8,20.4,20.4,21.9,20.8,72.0,41.0,56.0,34.0,22.0
2555,ROCK VALLEY SCHOOL DISTRICT,167736,2013.0,34,21.0,21.1,23.0,22.5,22.0,76.0,44.0,65.0,47.0,41.0
3491,ROCK VALLEY SCHOOL DISTRICT,167736,2019.0,34,18.7,19.5,20.6,20.6,19.9,53.0,35.0,35.0,35.0,18.0
3782,ROCK VALLEY SCHOOL DISTRICT,167736,2020.0,33,20.2,19.4,21.8,20.5,20.7,67.0,30.0,52.0,33.0,9.0
4904,ROCK VALLEY SCHOOL DISTRICT,167736,2021.0,26,21.5,20.5,23.6,22.6,22.2,77.0,38.0,58.0,46.0,27.0


To ensure our conversion worked, we will construct another frequency table that outputs the number of distinct names per district code. 

In [35]:
#observing frequency table after changes
dist_name_freq_post_changes = merged_act_data.groupby('ACT Dist Code')['District Name'].nunique().reset_index()
dist_name_freq_post_changes.rename(columns={'District Name': 'Distinct District Names'}, inplace=True)
dist_name_freq_post_changes.sort_values(by='Distinct District Names', ascending=False, inplace=True)

dist_name_freq_post_changes.head()

,ACT Dist Code,Distinct District Names
0,167004,1
205,167590,1
212,167604,1
211,167602,1
210,167600,1


The final step of data processing is to generate rows that calculate the average school performance in a given year. Important note, this is not a measure of average student performance, but of average school performance. Imagine if a 1000 students at school x took the act and averaged a 25, and 10 students took the act at school y and averaged 30. If those two schools were the only schools in  ur dataframe, then the statewide average would show 27.5, which would absolutely be an incorrect measure of average student performance. However, this might not be a terrible measure of average SCHOOL performance across the state.

In [36]:
def calc_state_avgs(df):

    """
    Calculate statewide averages for ACT data.

    This function takes a DataFrame containing ACT data for different school districts 
    and calculates the statewide averages for each year. It selects the relevant columns 
    containing ACT data, calculates the column averages grouped by graduation year, and 
    then concatenates these statewide averages with the original DataFrame.

    Parameters:
    - df (DataFrame): A DataFrame containing ACT data for different school districts.

    Returns:
    - DataFrame: A DataFrame with statewide averages concatenated with the original DataFrame.
                 The 'District Name' column for statewide averages is set to 'STATEWIDE AVERAGE' 
                 and the 'ACT Dist Code' column is set to 0.

    Example:
    >>> statewide_avgs_df = calc_state_avgs(act_data_df)
    """



    #select columns to average over
    act_data_cols = df.columns[4:]

    #calculate column averages, grouped by graduation year
    state_school_avgs = df.groupby('Grad Year')[act_data_cols].mean().reset_index()

    
    #for statewide averages, set district name to 'STATEWIDE AVERAGE' and code to 0
    #this will help us filter for it on our dashboard
    state_school_avgs['District Name'] = 'STATEWIDE AVERAGE'
    state_school_avgs['ACT Dist Code'] = 0

    #concat averages state averages together
    final_df = pd.concat([df, state_school_avgs], ignore_index=True)

    return final_df

#creaate final dataframe with statewide averates
final_act_df = calc_state_avgs(merged_act_data)

Visualizing the calculated rows:

In [43]:
statewide_avg_rows = final_act_df[final_act_df['District Name'] == 'STATEWIDE AVERAGE']


In [44]:
statewide_avg_rows.head(5)

,District Name,ACT Dist Code,Grad Year,N,Avg Eng,Avg Math,Avg Reading,Avg Sci,Avg Comp,CRB % Eng,CRB % Math,CRB % Reading,CRB % Sci,CRB % All Four
2892,STATEWIDE AVERAGE,0,2012.0,NaN,21.315947,21.375083,22.253156,22.036213,21.865449,77.109635,48.598007,60.355482,35.129568,26.601329
2893,STATEWIDE AVERAGE,0,2013.0,NaN,21.224092,21.196700,22.220462,21.958746,21.780198,75.851485,46.775578,51.016502,42.788779,27.498350
2894,STATEWIDE AVERAGE,0,2014.0,NaN,21.192491,21.096587,22.263823,22.073379,21.789761,75.174061,45.587031,50.494881,45.317406,28.095563
2895,STATEWIDE AVERAGE,0,2015.0,NaN,21.233677,21.131615,22.358419,22.064261,21.831615,74.680412,44.972509,52.549828,44.969072,28.838488
2896,STATEWIDE AVERAGE,0,2016.0,NaN,21.112500,21.013851,22.414527,22.058108,21.780743,73.243243,44.760135,52.537162,43.334459,28.216216


In [18]:
#export final excel file
final_act_df.to_excel('final_act.xlsx', index=False)